In [323]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4  import *
from dnn_utils_v2 import sigmoid , sigmoid_backward , relu , relu_backward


import scipy
from PIL import Image
from scipy import ndimage
import skimage
from dnn_utils_v2 import *

In [324]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0 , 4.0)
plt.rcParams['image.interpolation']= 'nearest'
plt.rcParams['image.cmap']= 'gray'

%reload_ext autoreload
%autoreload 2

np.random.seed(1)

In [305]:
#inicializar parametros
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    for i in range(1,L):
        parameters["W"+str(i)] =  np.random.randn(layer_dims[int(i)] ,layer_dims[int(i)-1]) *0.01
        parameters["b"+str(i)] =  np.zeros((layer_dims[int(i)] , 1))
        
        print("W" + str(i)  + " = " + str(parameters["W"+str(i)].shape))
        print("b" + str(i)  + " = " + str( parameters["b"+str(i)].shape))
        
        assert(parameters['W' + str(i)].shape == (layer_dims[int(i)], layer_dims[int(i)-1]))
        assert(parameters['b' + str(i)].shape == (layer_dims[int(i)], 1))
    return parameters


In [306]:
#Probando la función
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = (4, 5)
b1 = (4, 1)
W2 = (3, 4)
b2 = (3, 1)
W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [271]:
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    return Z, cache

In [272]:
#Probando la función
A, W, b = linear_forward_test_case()
Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [294]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    return A, cache

In [210]:
#Probando a la función
A_prev, W, b = linear_activation_forward_test_case()
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [211]:
def L_model_forward(X, parameters):
    
    caches =[]
    A=X;
    L=len(parameters)//2
    caches=list()
    
    # Implemente [LINEAR -> RELU]*(L-1). Agregue "cache" a la lista "caches"
    
    for l in range(1,L):
        A_prev = A
        A , cache = linear_activation_forward(A_prev, parameters["W" + str(l)] , parameters["b" + str(l)], activation = "relu")
        caches.append(cache)
        
     # Implementa LINEAR -> SIGMOID. Agregue "cache" a la lista "caches"
    AL, cache = linear_activation_forward(A , parameters["W" + str(L)] , parameters["b" + str(L)], activation = "sigmoid" )
    caches.append(cache)
    return AL, caches

In [308]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


In [313]:
def compute_cost (AL,Y):
    m = Y.shape[1]
    
    cost = ( np.sum( np.dot( Y,np.log(AL).T ) + np.dot( ( 1-Y ),( np.log(1-AL).T ) ) ) )/(-m)
    
    cost = np.squeeze(cost)
    assert( cost.shape == () )
    return cost

In [314]:
Y, AL = compute_cost_test_case()
print("cost = " + str(compute_cost(AL, Y)))

cost = 0.414931599615397


In [287]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = ( np.dot( dZ,A_prev.T) )/(m)
    db = ( np.sum(dZ , axis=1, keepdims=True) )/(m)
    dA_prev = np.dot( W.T,dZ )

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [249]:
#Probando la función
dZ, linear_cache = linear_backward_test_case()
dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))


dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


In [250]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ =  relu_backward (dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation == "sigmoid":
        dZ = sigmoid_backward (dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [251]:
#Probando la función
dAL, linear_activation_cache = linear_activation_backward_test_case()
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [315]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # Iniciando el backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    # en cada capa l (SIGMOID -> LINEAR) gradients. Entrada: "dAL, current_cache". Salida: 
    # "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1] 
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward( dAL, current_cache, "sigmoid" )

    # Loop de l=L-2 to l=0
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)],current_cache,"relu")
    
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [316]:
#Probando la función
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL , Y_assess , caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [317]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # numero de capas de la red
    for l in range(L):
        parameters["W" + str(l+1)] = (parameters["W"+ str(l+1)]  - learning_rate*grads["dW"+ str(l+1)])
        parameters["b" + str(l+1)] = (parameters["b"+ str(l+1)]  - learning_rate*grads["db"+ str(l+1)])
    return parameters

In [255]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)
print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


In [318]:
def nn_model(X, Y, layer_dims , num_iterations = 10000, learning_rate=1.2 , print_cost=False):
    
    np.random.seed(1)
    
    parameters = initialize_parameters_deep(capas)
    costo=[]
    
    for i in range(num_iterations):
        #FORWARD PROGATION
        AL , caches = L_model_forward(X, parameters) 
        
        #CALCULA FUNCION COSTO
        cost = compute_cost(AL , Y)
        
        #BACK PROGATION
        grads = L_model_backward(AL, Y, caches)
        
        #ACTUALIZACION DE PESOS 
        parameters = update_parameters(parameters, grads , learning_rate)
        
        if print_cost and i % 10 == 0:
            print ("Costo despues de iteracion: ",i," = ", cost)
        if  i % 1000 == 0:
            costo.append(cost)
    return parameters

In [319]:
def predict(parameters, X):
    A2, cache = L_model_forward(X, parameters)
    predictions = np.rint(A2)
    return predictions

In [331]:
#OBTECION DE DATOS
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])
    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])
    classes = np.array(test_dataset["list_classes"][:])
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))#---
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [332]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_dataset()
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T  
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten/255.
test_x = test_x_flatten/255.


In [321]:
capas = [12288,1]

In [333]:
parameters  = nn_model(train_set_x,  train_y , layer_dims  , 10000 , 1.3 , print_cost=True )

W1 = (1, 12288)
b1 = (1, 1)
Costo despues de iteracion:  0  =  0.6909085741648417


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
C:\Users\Lenovo\UNA\REDES PROFUNDAS\RED-NUERONAL-PROFUNDA\dnn_utils_v2.py:77: RuntimeWarning: invalid value encountered in multiply
  dZ = dA * s * (1-s)


Costo despues de iteracion:  10  =  nan
Costo despues de iteracion:  20  =  nan
Costo despues de iteracion:  30  =  nan
Costo despues de iteracion:  40  =  nan
Costo despues de iteracion:  50  =  nan
Costo despues de iteracion:  60  =  nan
Costo despues de iteracion:  70  =  nan
Costo despues de iteracion:  80  =  nan
Costo despues de iteracion:  90  =  nan
Costo despues de iteracion:  100  =  nan
Costo despues de iteracion:  110  =  nan
Costo despues de iteracion:  120  =  nan
Costo despues de iteracion:  130  =  nan
Costo despues de iteracion:  140  =  nan
Costo despues de iteracion:  150  =  nan
Costo despues de iteracion:  160  =  nan
Costo despues de iteracion:  170  =  nan
Costo despues de iteracion:  180  =  nan
Costo despues de iteracion:  190  =  nan
Costo despues de iteracion:  200  =  nan
Costo despues de iteracion:  210  =  nan
Costo despues de iteracion:  220  =  nan
Costo despues de iteracion:  230  =  nan
Costo despues de iteracion:  240  =  nan
Costo despues de iteracio

KeyboardInterrupt: 